Arquivo de auxílio para implementação de códigos responsáveis por auxiliar na documentação do repositório

In [1]:
# Importando bibliotecas
import pandas as pd
import json
import os

In [6]:
# Definindo variáveis
DATA_PATH = '../data'

# Dicionário de descrição de metadados de bases
ACTIVITY_DATA_METADATA = [
    'Tempo de chegada do usuário',
    'Tempo de criação da medição sensorial',
    'Identificação do dispositivo de medição',
    'Índice do dispositivo de medição',
    'Modelo do dispositivo de medição',
    'Identificação do usuário',
    'Categoria',
    'Coordenada x da medição do usuário',
    'Coordenada y da medição do usuário',
    'Coordenada z da medição do usuário'
]

In [5]:
# Obtendo tamanho e quantidade de arquivos de um diretório
def get_data_dir_info(data_path=DATA_PATH):
    # Iterando sobre diretórios de dados na pasta raíz
    df_data_info = pd.DataFrame()
    for dir in [d for d in os.listdir(data_path) if '.' not in d]:
        # Extraindo iinformações do diretório
        dir_path = os.path.join(data_path, dir)
        qtd_files = len(os.listdir(dir_path))
        nbytes = sum(d.stat().st_size for d in os.scandir(dir_path) if d.is_file())
        nkilos = round(nbytes / (1024), 2)
        nmegas = round(nbytes / (1024 * 1024), 2)
        ngigas = round(nbytes / (1024 * 1024 * 1024), 2)
        
        # Appendando no DataFrame
        row_info = {
            'dir': dir,
            'qtd_files': qtd_files,
            'nbytes': nbytes,
            'nkilos': nkilos,
            'nmegas': nmegas,
            'ngigas': ngigas
        }
        df_data_info = df_data_info.append(row_info, ignore_index=True)

    return df_data_info

df_data_info = get_data_dir_info()
df_data_info

,dir,qtd_files,nbytes,nkilos,nmegas,ngigas
0,activity-data,80.0,1.192764e+09,1164808.76,1137.51,1.11
1,airbnb,1.0,3.423464e+07,33432.26,32.65,0.03
2,bike-data,2.0,0.000000e+00,0.00,0.00,0.00
3,blogs,1.0,9.210000e+02,0.90,0.00,0.00
4,employee,4.0,0.000000e+00,0.00,0.00,0.00
5,flights-data,3.0,0.000000e+00,0.00,0.00,0.00
6,iot-devices,1.0,6.270722e+07,61237.52,59.80,0.06
7,loans,1.0,1.646310e+05,160.77,0.16,0.00
8,retail-data,1.0,4.558067e+07,44512.37,43.47,0.04
9,sf-fire,2.0,0.000000e+00,0.00,0.00,0.00


___

In [33]:
# Função para ler amostra de dados
def read_sample(file_path, file_ext):

    # Lendo arquivo json
    if file_ext == '.json':
        with open(file_path, 'rb') as f:
            sample = f.readline().decode()
            json_sample = json.loads(sample)

        # Transformando json em DataFrame
        df_sample = pd.DataFrame(json_sample, index=['value']).T

    # Lendo arquivo csv
    elif file_ext == '.csv':
        df_sample = pd.read_csv(file_path, nrows=1).T
        df_sample.columns = ['value']
        
    # Resetando index e padronizando DataFrame
    df_sample.reset_index(drop=False, inplace=True)
    
    # Enriquecendo DataFrame
    df_sample['dtype'] = [type(row) for row in df_sample['value'].values]
    df_sample.columns = ['Coluna', 'Exemplo', 'Tipo Primitivo']

    return df_sample

# Função para transformar schema de amostra em código markdown de tabela para doc
def transform_schema_to_markdown_table(sample_file_path, metadata):

    # Extraindo tipo primitivo para validação da leitura
    file_ext = os.path.splitext(os.path.basename(sample_file_path))[-1]

    # Obtendo amostra em formato DataFrame
    df_sample = read_sample(file_path=sample_file_path, file_ext=file_ext)

    # Adicionando descrição
    df_sample['Descrição'] = metadata

    # Reordenando colunas
    cols_order = ['Coluna', 'Descrição', 'Tipo Primitivo', 'Exemplo']
    df_sample = df_sample.loc[:, cols_order]

    # Transformando colunas em markdown
    header_mkdown = ''.join([' | ' + c for c in df_sample.columns])[1:] + ' |'
    header_sec_mkdown = ''.join([' | :---:'] * len(df_sample.columns))[1:] + ' |'

    # Transformando registros em markdown
    row_mkdown_str = []
    for idx, row in df_sample.iterrows():
        row_mkdown_str.append(''.join([' | ' + str(v) for v in dict(row).values()])[1:] + ' |')
    row_mkdown = '\n'.join(row_mkdown_str)

    # Unindo headers com registros
    table_mkdown = header_mkdown + '\n' + header_sec_mkdown + '\n' + row_mkdown

    return table_mkdown
    

In [34]:
FOLDER_PATH = os.path.join(DATA_PATH, 'activity-data')
SAMPLE_FILE = os.path.join(FOLDER_PATH, os.listdir(FOLDER_PATH)[0])
print(transform_schema_to_markdown_table(sample_file_path=SAMPLE_FILE, metadata=ACTIVITY_DATA_METADATA))

| Coluna | Descrição | Tipo Primitivo | Exemplo |
| :---: | :---: | :---: | :---: |
| Arrival_Time | Tempo de chegada do usuário | <class 'int'> | 1424686735175 |
| Creation_Time | Tempo de criação da medição sensorial | <class 'int'> | 1424686733176178965 |
| Device | Identificação do dispositivo de medição | <class 'str'> | nexus4_1 |
| Index | Índice do dispositivo de medição | <class 'int'> | 35 |
| Model | Modelo do dispositivo de medição | <class 'str'> | nexus4 |
| User | Identificação do usuário | <class 'str'> | g |
| gt | Categoria | <class 'str'> | stand |
| x | Coordenada x da medição do usuário | <class 'float'> | 0.0014038086 |
| y | Coordenada y da medição do usuário | <class 'float'> | 0.00050354 |
| z | Coordenada z da medição do usuário | <class 'float'> | -0.0124053955 |


___

In [35]:
# Testando execução em loop
DATA_FOLDERS = os.listdir(DATA_PATH)[:2]

# Iterando sobre diretórios de arquivos
mkdown_table_dict = {}
for dir in DATA_FOLDERS:
    dir_path = os.path.join(DATA_PATH, dir)
    sample = os.path.join(dir_path, os.listdir(dir_path)[0]) # Validar se este elemento é um diretório. Se sim, iterar nele e pegar amostras
    mkdown_table_dict[dir_path] = transform_schema_to_markdown_table(sample_file_path=sample, metadata='Teste')

In [39]:
print(mkdown_table_dict['../data\\airbnb'])

| Coluna | Descrição | Tipo Primitivo | Exemplo |
| :---: | :---: | :---: | :---: |
| id | Teste | <class 'int'> | 958 |
| listing_url | Teste | <class 'str'> | https://www.airbnb.com/rooms/958 |
| scrape_id | Teste | <class 'int'> | 20190306152813 |
| last_scraped | Teste | <class 'str'> | 2019-03-06 |
| name | Teste | <class 'str'> | Bright, Modern Garden Unit - 1BR/1B |
| summary | Teste | <class 'str'> | New update: the house next door is under construction and there is the possibility of noise from 7am - 5pm. Our rates are discounted during this time period. Our bright garden unit overlooks a grassy backyard area with fruit trees and native plants. It is an oasis in a big city. The apartment comfortably fits a couple or small family. It is located on a cul de sac street that ends at lovely Duboce Park. |
| space | Teste | <class 'str'> | Newly remodeled, modern, and bright garden unit in historic Victorian home.  *New fixtures and finishes. *Organic cotton sheets and towels. *Zero

In [32]:
df_sample = pd.read_csv(sample, nrows=1).T
df_sample.columns = ['value']
df_sample.reset_index(drop=False, inplace=True)
df_sample

,index,value
0,id,958
1,listing_url,https://www.airbnb.com/rooms/958
2,scrape_id,20190306152813
3,last_scraped,2019-03-06
4,name,"Bright, Modern Garden Unit - 1BR/1B"
...,...,...
101,calculated_host_listings_count,1
102,calculated_host_listings_count_entire_homes,1
103,calculated_host_listings_count_private_rooms,0
104,calculated_host_listings_count_shared_rooms,0
